In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten
import pandas as pd
import numpy as np

2025-01-27 18:29:55.917285: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 18:29:56.091955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738002596.167896    2527 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738002596.188321    2527 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 18:29:56.352796: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Load dataset
df = pd.read_csv("balanced_soldier_data.csv")

In [4]:
df.head()

,Temperature,Moisture,Water_Content,SpO2,Fatigue,Drowsiness,Stress,Heart_Rate,Respiration_Rate,Systolic_BP,Diastolic_BP,Efficiency
0,35.383229,65.701296,74.896995,96.144655,0.208223,0.078256,0.167958,66.431073,15.907304,119,78,0.726903
1,36.028752,69.182477,87.289935,99.285356,0.186550,0.138735,0.008335,82.207135,17.644151,113,76,0.775944
2,37.004390,38.142130,51.690862,88.401253,0.795877,0.860479,0.821897,115.268466,25.442539,137,88,0.216759
3,35.926560,56.353209,86.145719,97.281401,0.074097,0.278761,0.125591,86.724807,15.603473,117,70,0.719477
4,36.231738,46.817432,48.037744,89.566148,0.779676,0.624617,0.829127,118.251208,23.152386,135,85,0.247567


In [9]:
df["Squad Efficiency"] = df.groupby(df.index // 10)["Efficiency"].transform("mean")

In [13]:
# Select relevant features
features = [
    "Temperature",
    "Moisture",
    "Water_Content",
    "SpO2",
    "Fatigue",
    "Drowsiness",
    "Stress",
    "Heart_Rate",
    "Respiration_Rate",
    "Systolic_BP",
    "Diastolic_BP",
    "Squad Efficiency",
]
target = "Strike Success Probability"  # This will be our label

In [14]:
# Generate synthetic labels (for demonstration)
df[target] = df["Squad Efficiency"].apply(
    lambda x: np.random.uniform(0, 1) if x > 0.6 else np.random.uniform(0, 0.5)
)

In [15]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [16]:
X = df[features].values
y = df[target].values

In [17]:
# Reshape input for LSTM (Samples, Time Steps, Features)
X = X.reshape((X.shape[0] // 10, 10, X.shape[1]))  # Grouping 10 soldiers per second
y = y[::10]  # One label per second

In [18]:
# Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [20]:
def build_model():
    # Build the Deep Learning Model
    model = Sequential(
        [
            LSTM(64, return_sequences=True, input_shape=(10, len(features))),
            Dropout(0.2),
            LSTM(32, return_sequences=False),
            Dropout(0.2),
            Dense(16, activation="relu"),
            Dense(8, activation="relu"),
            Dense(1, activation="sigmoid"),  # Output between 0 and 1
        ]
    )
    return model

In [21]:
model = build_model()

/root/miniconda3/envs/myenv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [23]:
# Train the model
history = model.fit(
    X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test)
)

Epoch 1/50


I0000 00:00:1738003002.961305    3751 cuda_dnn.cc:529] Loaded cuDNN version 90300


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.0000e+00 - loss: 0.6735 - val_accuracy: 0.0000e+00 - val_loss: 0.5767
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.0000e+00 - loss: 0.5802 - val_accuracy: 0.0000e+00 - val_loss: 0.5573
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.0000e+00 - loss: 0.5731 - val_accuracy: 0.0000e+00 - val_loss: 0.5567
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.0000e+00 - loss: 0.5774 - val_accuracy: 0.0000e+00 - val_loss: 0.5557
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.0000e+00 - loss: 0.5665 - val_accuracy: 0.0000e+00 - val_loss: 0.5545
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - accuracy: 0.0000e+00 - loss: 0.5642 - val_accuracy: 0.0000e+00 - val_loss: 0.5559
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.0000e+00 - loss: 0.5707 - val_accuracy: 0.0000e+00 - val_loss: 0.5553
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.0

In [24]:
# Save the model
model.save("TrainedModel/surgical_strike_success_model.keras")

In [25]:
# Test the model
test_prediction = model.predict(X_test[:5])
print("Predicted Success Chances:", test_prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Predicted Success Chances: [[0.26711515]
 [0.27600867]
 [0.26933897]
 [0.26997417]
 [0.2754047 ]]


In [26]:
import joblib

# Save the fitted scaler
joblib.dump(scaler, "TrainedModel/surgicalScaler.pkl")
print("Scaler saved successfully.")

Scaler saved successfully.
